## With this notebook; 

* I want to keep up my DataCamp Tracks, I created the dataset by the courses which I am interested in individually including track information and put dataset on my Google Drive acoount. For reading Google spreadsheet; after create project and Api Authentication(Google Sheet API and Google Drive API), I started to read from drive excel.

* My goal is to follow up on this file and work on the current file instead of a fixed dataset so that I can follow my progress.
* While I was developing my Data Science skills with the DataCamp course, this training time also will be a project for me :)

In [8]:
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [10]:
import gspread

gc=gspread.oauth()
sheet1 = gc.open("datacamp").sheet1

data = sheet1.get_all_values()
headers = data.pop(0)


In [12]:

import pandas as pd
df = pd.DataFrame(data, columns=headers)

** pre_processing data, NaN values, dtypes **

In [14]:
df.replace('', 0, inplace=True)
df['Planning Date']=pd.to_datetime(df['Planning Date'])

col_dict={'No':int, 'order':int,'Watch':int,'Learn':int,'Github':int,'Video':int,'Pdf':int}
for k, v in col_dict.items():
    df[k]=df[k].astype(v)
print(df.dtypes)

No                        int32
Track                    object
Type                     object
order                     int32
Course                   object
Planning Date    datetime64[ns]
Watch                     int32
Learn                     int32
Github                    int32
Video                     int32
Pdf                       int32
dtype: object


** define necessary methods for statisticals **

In [ ]:
def remain(x):
    return x.count() - x.sum()
remain.__name__ = 'Remain'

def percent(x):
    return x.sum()*100 / x.count()
percent.__name__='Comp. %'

** some courses take place in multiple tracks. Once you complete one, others must be update as completed  as well. **

In [6]:
from collections import Counter
course_count= Counter(df.Course)
for course in course_count.most_common():
    print(course)


('Intermediate Python', 5)
('Python Data Science Toolbox (Part 1)', 5)
('Python Data Science Toolbox (Part 2)', 5)
('Data Manipulation with pandas', 4)
('Introduction to Data Visualization with Seaborn', 4)
('Introduction to Python', 4)
('Machine Learning with Tree-Based Models in Python', 4)
('Supervised Learning with scikit-learn', 4)
('Unsupervised Learning in Python', 4)
('Writing Efficient Python Code', 4)
('Writing Functions in Python', 4)
('Analyzing Police Activity with pandas', 3)
('Cleaning Data in Python', 3)
('Cluster Analysis in Python', 3)
('Intermediate Importing Data in Python', 3)
('Introduction to Data Visualization with Matplotlib', 3)
('Introduction to Deep Learning in Python', 3)
('Introduction to Importing Data in Python', 3)
('Introduction to PySpark', 3)
('Introduction to Relational Databases in SQL', 3)
('Introduction to Shell', 3)
('Joining Data with pandas', 3)
('Object-Oriented Programming in Python', 3)
('Statistical Thinking in Python (Part 1)', 3)
('Unit 

In [7]:
update_columns={'Watch':'max', 'Learn':'max', 'Github':'max', 'Video':'max','Pdf':'max','Planning Date':'min'}
max_=df.groupby('Course').agg(update_columns)
for key, row_ in zip(max_.index, max_.values):
    df.loc[df.Course==key, update_columns.keys()]=row_

** summary table for completence of the course, skill tracks **

In [9]:
statistics={
    'Learn':['sum','count', remain, percent], 
                         'Github':'sum', 
                         'Video':'sum',
                         'Pdf':'sum',
                         'Planning Date':'min'}
df_stats= df.sort_values(['Type','No','Track'],ascending=False).groupby(['Type','No','Track'], as_index=False).agg(statistics)
df_stats.columns = [ "_".join(x) if x[0]=='Learn' else x[0] for x in df_stats.columns.ravel() ]
# sum_learn=('learn',sum)
df_stats


,Type,No,Track,Learn_sum,Learn_count,Learn_Remain,Learn_Comp. %,Github,Video,Pdf,Planning Date
0,Career,1,Python Programmer,5,16,11,31.250000,7,0,0,2020-09-16
1,Career,2,Data Analyst with Python,2,16,14,12.500000,2,0,0,2020-09-17
2,Career,3,Data Scientist with Python,5,23,18,21.739130,7,0,0,2020-09-16
3,Career,4,Data Science for Everyone,5,26,21,19.230769,5,0,0,2020-09-16
4,Career,5,Machine Learning for Everyone,4,29,25,13.793103,4,0,0,2020-09-16
5,Career,6,Machine Learning Scientist with Python,0,23,23,0.000000,0,0,0,2020-11-04
6,Career,7,Data Engineer with Python,1,25,24,4.000000,2,0,0,2020-09-20
7,Course,18,Natural Language Processing in Python,0,2,2,0.000000,0,0,0,2021-03-03
8,Course,19,Marketing Analytics with Python,0,2,2,0.000000,0,0,0,2021-03-10
9,Course,20,Big Data with PySpark,0,1,1,0.000000,0,0,0,2021-03-12



for col in enumerate(df_stats.index):
    #<class 'tuple'> . 2 . (0, ('Career', 1, 'Python Programmer')) . 0 . ('Career', 1, 'Python Programmer') . Career
    df_stats.loc[col[1]]['my_index']=col[1]
    #print(type(col), '.',len(col), '.',col,'.',col[0],'.',col[1],'.', col[1][0])
    #print( df_stats.loc[col[1]])

for col in enumerate(df_stats.columns):
    print(type(col), '.',len(col), '.',col,'.',col[0],'.',col[1],'.', col[1][1], df_stats[col[1]].values[0])
print((df_stats.columns)[0][1])
df_stats['my_index2']=(df_stats.columns)[1][1]

In [11]:
df_stats[['day','month','year']] = df_stats['Planning Date'].apply(lambda x: pd.Series([x.day,x.month,x.year]))


In [13]:
from datetime import datetime
import plotly.express as px
fig = px.bar(df_stats[df_stats['Planning Date'].dt.year==2020], x=["Learn_count", "Learn_sum" ], y='Track',  color='Type')
fig.show()

In [15]:
fig.to_image("images/f.png")

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    C:\Program Files (x86)\Common Files\Oracle\Java\javapath
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files\Microsoft SQL Server\120\Tools\Binn\
    C:\Program Files (x86)\Microsoft SQL Server\110\Tools\Binn\
    C:\Program Files\Microsoft SQL Server\110\Tools\Binn\
    C:\Program Files\Microsoft SQL Server\110\DTS\Binn\
    C:\Program Files\Microsoft SQL Server\Client SDK\ODBC\110\Tools\Binn\
    C:\Program Files (x86)\Microsoft SQL Server\120\Tools\Binn\
    C:\Program Files\Microsoft SQL Server\120\DTS\Binn\
    C:\Program Files\nodejs\
    C:\Program Files\dotnet\
    C:\Program Files\Microsoft SQL Server\130\Tools\Binn\
    C:\Program Files\Microsoft SQL Server\Client SDK\ODBC\170\

** This is dictionary of career tracks including courses with order number **

In [17]:
career_tracks= list(map( lambda Track :{Track: dict(zip(df[df['Track']==Track].order, df[df['Track']==Track]['Course']))}, df[df['Type']=='Career'].Track))

In [18]:
for career in career_tracks[:5]:
    print(career)

{'Machine Learning for Everyone': {23: 'Advanced Deep Learning with Keras', 12: 'Cluster Analysis in Python', 13: 'Dimensionality Reduction in Python', 11: 'Extreme Gradient Boosting with XGBoost', 16: 'Feature Engineering for Machine Learning in Python', 19: 'Feature Engineering for NLP in Python', 26: 'Hyperparameter Tuning in Python', 24: 'Image Processing in Python', 25: 'Image Processing with Keras in Python', 3: 'Intermediate Python', 21: 'Introduction to Deep Learning in Python', 22: 'Introduction to Deep Learning with Keras', 18: 'Introduction to Natural Language Processing in Python', 27: 'Introduction to PySpark', 2: 'Introduction to Python', 20: 'Introduction to TensorFlow in Python', 9: 'Linear Classifiers in Python', 1: 'Machine Learning for Everyone', 15: 'Machine Learning for Time Series Data in Python', 28: 'Machine Learning with PySpark', 10: 'Machine Learning with Tree-Based Models in Python', 17: 'Model Validation in Python', 14: 'Preprocessing for Machine Learning i

In [19]:

from collections import Counter
learn_count= Counter(df.Learn)
for course in learn_count.most_common(5):
    print('.',course,'.')

. (0, 182) .
. (1, 26) .


In [20]:
print(sheet1.row_values(1))
#dir(sheet1)


['No', 'Track', 'Type', 'order', 'Course', 'Planning Date', 'Watch', 'Learn', 'Github', 'Video', 'Pdf']
